Objetivo General

Aprender a realizar un análisis integral de un portafolio financiero  utilizando yfinance, comprendiendo los conceptos de rendimiento, riesgo, correlación y optimización, para aplicarlos luego en una aplicación Streamlit que visualice los resultados.

Cada ejercicio de este taller representa un análisis que luego formará parte de la aplicación interactiva:

| Ejercicio | Tipo de análisis | Aplicación en Streamlit |
|------------|------------------|--------------------------|
| 1 | Exploración de precios | Gráfico de precios por ticker |
| 2 | Rendimientos | Gráfico de crecimiento acumulado |
| 3 | Correlación | Heatmap de correlaciones |
| 4–5 | Portafolios básicos | Comparador de rendimiento y riesgo |
| 6 | Frontera eficiente | Visualización interactiva |
| 7 | Ratio de Sharpe | Tabla con desempeño ajustado al riesgo |
| 8 | Backtesting | Gráfico por año |
| 9 | Benchmarking | Comparación con S&P 500 |
| 10 | Monte Carlo | Simulación de escenarios |

---

# Contexto de la historia

Eres un analista financiero recién contratado por FinanSmart Capital, una startup de inversión que busca crear una aplicación de análisis de portafolios inteligente.
Tu misión es evaluar la rentabilidad y el riesgo de un conjunto de acciones tecnológicas y proponer un portafolio óptimo que maximice los rendimientos minimizando el riesgo.
Cada paso de este taller representa una etapa de análisis que será parte del desarrollo del módulo analítico de la aplicación Streamlit.

## 🕵️‍♀️ Misión 1: Conociendo a tus activos

FinanSmart te ha pedido empezar con un grupo de acciones tecnológicas líderes del mercado:
AAPL, MSFT, AMZN, GOOGL, META.

Tu tarea es descargar sus precios históricos de los últimos 3 años.

🧠 ¿Por qué esto es importante?
El análisis histórico te permitirá comprender el comportamiento de cada activo, su tendencia y su volatilidad.

In [ ]:
import yfinance as yf  # Librería para obtener datos financieros directamente desde Yahoo Finance

# Definimos los tickers (símbolos bursátiles) de las acciones que formarán el portafolio
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META']

# Descargamos los precios ajustados de cierre (Adjusted Close)
# Estos precios corrigen splits y dividendos, lo que los hace adecuados para análisis financiero
data = yf.download(tickers, start='2020-01-01', end='2023-12-31')['Close']

# Visualizamos las primeras filas para verificar la estructura de los datos
data.head()



## Teoría aplicada:

Esta etapa corresponde a la definición del universo de inversión. En la teoría de Markowitz, el primer paso es seleccionar los activos que formarán parte del portafolio.

## 📊 Misión 2: Visualizando el pasado

Tu supervisor quiere ver cómo han evolucionado las acciones a lo largo del tiempo.

🎯 Meta: Crear un gráfico de precios ajustados para presentar en la próxima reunión.

In [ ]:
import matplotlib.pyplot as plt

data.plot(figsize=(10,6), title='Evolución de precios ajustados')
plt.show()


Aquí se analiza el comportamiento histórico de los activos, base para estimar su rendimiento esperado y riesgo, componentes fundamentales en la frontera eficiente de Markowitz.

## 💹 Misión 3: Midiendo el pulso del mercado

Ahora necesitas calcular los retornos diarios para entender cómo se comportan las acciones en el día a día.

🧠 Reflexión: Los retornos muestran las variaciones de precio — un paso clave para evaluar el riesgo y la rentabilidad.

In [ ]:
# Calculamos el retorno porcentual diario
returns = data.pct_change().dropna()

# Mostramos estadísticas descriptivas (media, desviación, etc.)
returns.describe()



## Teoría aplicada:
El retorno es la variable central del modelo de Markowitz. Se utiliza para calcular el rendimiento esperado (media) y el riesgo (desviación estándar o varianza) del portafolio.

## ⚡ Misión 4: Detectando la volatilidad

Visualiza los retornos diarios. Observa cuál acción parece más volátil (mayor amplitud de variaciones).

💬 Tip: Una alta volatilidad puede significar tanto oportunidad como riesgo.

In [ ]:
returns.plot(figsize=(10,6), alpha=0.7, title='Retornos diarios')
plt.show()


## Teoría aplicada:
La volatilidad representa el riesgo individual de cada activo. En la teoría moderna de portafolios, el riesgo se mide mediante la varianza o desviación estándar de los retornos.

## 🤝 Misión 5: El juego de las correlaciones

Tu equipo necesita saber si las acciones se mueven en conjunto o de forma independiente.
Crea una matriz de correlación para descubrir las relaciones entre ellas.

📈 Insight: Si las acciones están poco correlacionadas, diversificar el portafolio reducirá el riesgo.

In [ ]:
import seaborn as sns
sns.heatmap(returns.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de correlación del portafolio')
plt.show()


## Teoría aplicada:
Markowitz plantea que la diversificación reduce el riesgo si los activos no están perfectamente correlacionados.
Por eso la matriz de correlación (o covarianza) es esencial: mide cómo se mueven los activos juntos.

## 📈 Misión 6: Calculando riesgo y rendimiento

Ahora calcula el rendimiento anual esperado y la volatilidad de cada acción.

📊 Esto servirá para construir la base del análisis de portafolio.

In [ ]:
import numpy as np

# Rendimiento medio diario multiplicado por 252 días hábiles del año
mean_returns = returns.mean() * 252

# Desviación estándar diaria multiplicada por la raíz de 252 para anualizar el riesgo
risk = returns.std() * np.sqrt(252)

# Mostramos ambos valores en una tabla
pd.DataFrame({'Rendimiento Anual': mean_returns, 'Riesgo (Volatilidad)': risk})



## Teoría aplicada:
Estos dos indicadores forman el eje del modelo media-varianza de Markowitz:

Media de retornos → rendimiento esperado.

Desviación estándar → riesgo total.

## 💰 Misión 7: Creando un portafolio balanceado

Comienza con un portafolio con pesos iguales para todas las acciones.

🎯 Objetivo: Calcular el rendimiento y riesgo del portafolio conjunto.

In [ ]:
# Asignamos pesos iguales a cada activo
weights = np.array([1/len(tickers)] * len(tickers))

# Rendimiento esperado del portafolio = suma ponderada de los rendimientos individuales
portfolio_return = np.dot(weights, mean_returns)

# Riesgo del portafolio considerando la covarianza entre activos
portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))

print(f'Rendimiento esperado del portafolio: {portfolio_return:.2%}')
print(f'Riesgo del portafolio: {portfolio_risk:.2%}')



Teoría aplicada:
Se aplica la fórmula de rendimiento esperado y riesgo del portafolio según Markowitz:

𝐸
(
𝑅
𝑝
)
=
∑
𝑤
𝑖
𝐸
(
𝑅
𝑖
)
E(R
p
	​

)=∑w
i
	​

E(R
i
	​

)

𝜎
𝑝
=
𝑤
𝑇
Σ
𝑤
σ
p
	​

=
w
T
Σw
	​


donde
Σ
Σ es la matriz de covarianzas.

## ⚖️ Misión 8: Explorando miles de escenarios

Tu jefe quiere ver la frontera eficiente. Simula 10.000 combinaciones de portafolios aleatorios.

🚀 Propósito: Encontrar combinaciones que maximizan retorno con el menor riesgo.

In [ ]:
num_portfolios = 10000  # Número de portafolios a simular
results = np.zeros((3, num_portfolios))  # Matriz para almacenar riesgo, retorno y Sharpe

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))  # Generamos pesos aleatorios
    weights /= np.sum(weights)  # Normalizamos para que sumen 1
    ret = np.dot(weights, mean_returns)  # Rendimiento esperado
    risk = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))  # Riesgo total
    sharpe = ret / risk  # Índice de Sharpe
    results[0,i] = risk
    results[1,i] = ret
    results[2,i] = sharpe

# Visualizamos la frontera eficiente
plt.scatter(results[0,:], results[1,:], c=results[2,:], cmap='viridis', alpha=0.5)
plt.xlabel('Riesgo (Volatilidad)')
plt.ylabel('Retorno Esperado')
plt.title('Frontera Eficiente Simulada')
plt.colorbar(label='Índice de Sharpe')
plt.show()



## Teoría aplicada:
Aquí se construye empíricamente la Frontera Eficiente de Markowitz, el conjunto de portafolios con el máximo retorno posible para un nivel dado de riesgo.

## 🏆 Misión 9: Encontrando el portafolio ganador

Identifica cuál combinación ofrece el mejor índice de Sharpe (mayor rendimiento por unidad de riesgo).

In [ ]:
max_sharpe_idx = np.argmax(results[2])
mejor_riesgo, mejor_retorno, mejor_sharpe = results[:, max_sharpe_idx]

print(f'Mejor Índice de Sharpe: {mejor_sharpe:.2f}')
print(f'Retorno esperado: {mejor_retorno:.2%}')
print(f'Riesgo asociado: {mejor_riesgo:.2%}')


El Índice de Sharpe mide la eficiencia del portafolio:

## 🧾 Misión 10: Preparando los datos para tu app Streamlit

Guarda los resultados para cargarlos en tu aplicación interactiva.

In [ ]:
df_resultados = pd.DataFrame(results.T, columns=['Riesgo', 'Retorno', 'Sharpe'])
df_resultados.to_csv('resultados_portafolio.csv', index=False)
df_resultados.head()


🏁 Reflexión Final

Has completado el desafío FinanSmart Capital 🎯
Ahora puedes usar estos resultados para alimentar tu aplicación en Streamlit, mostrando:

📈 Evolución de precios

💹 Rendimientos y riesgos

🤝 Correlaciones


A lo largo de este taller se aplicaron los pilares de la Teoría Moderna de Portafolios de Markowitz (1952):

Selección de activos.

* Cálculo de rendimientos esperados y riesgos individuales.

* Estimación de covarianzas y correlaciones.

* Construcción del portafolio.

* Determinación de la frontera eficiente.

* Optimización mediante el Índice de Sharpe.